# Knowledge Graph Retrieval using Amazon Neptune and LlamaIndex

## Introduction

In this notebook we are going to demonstrate how you can leverage [LlamaIndex](https://www.llamaindex.ai/), and specifically the [Property Graph Index](https://docs.llamaindex.ai/en/stable/examples/property_graph/property_graph_basic/) feature to perform knowledge graph retrieval with Amazon Neptune.  Let's start by taking a look at the two main components here: LlamaIndex and Knowledge Graph Retrieval.

LlamaIndex is a data structure and tooling designed to create and interact with large language model indexes. It facilitates the storage, searching, and querying of textual data using advanced vector database techniques in conjunction with large language models like GPT. This enables efficient and effective retrieval of relevant information from extensive text corpora.

Natural language querying allows users to interact with computer systems using everyday human language, rather than having to learn and use structured query languages or complex programming commands. This capability enables users to ask questions or provide instructions in their native tongue, and the system can then process this input to understand the user's intent and provide relevant information or perform the requested action.

However, in many real-world applications, it is not safe or appropriate to allow users to ask completely open-ended questions of a data set. For example, in a banking application, you may want to give users the ability to ask about their own transactions and account balance, but you certainly wouldn't want to allow them to inquire about other users' financial information.

To address these types of use cases, we can leverage a technique called Knowledge Graph Retrieval. Here, a large language model (LLM) is used to extract key entities from the user's natural language question. These extracted entities are then used as the parameters for a pre-defined, templated query. This approach gives the application developer the freedom to create optimized, secure queries with appropriate data access controls, while still providing users with a natural language interface on top of the underlying data.

### Prerequisites

For this notebook we will be using Amazon Neptune Database as our data store so you must have a Neptune Database configured.  The methodology presented here will also work with Neptune Analytics and we will call out where the code differs.  This notebook will also require permissions to run Amazon Bedrock models, specifically `Claude v3 Sonnet` and `Titan Embedding v1`.

### Installing our dependencies

Run the next cell to install the core LlamaIndex packages as well as the specific package for Amazon Neptune and Amazon Bedrock

In [1]:
%pip install -q llama-index llama-index-graph-stores-neptune llama-index-llms-bedrock llama-index-embeddings-bedrock 

Note: you may need to restart the kernel to use updated packages.


### Loading your Data

The data we will use in this notebook is based on the data in *[Graph Databases in Action](https://www.manning.com/books/graph-databases-in-action?a_aid=bechberger)* by Manning Publications. The book uses the most common graph access patterns to build a fictitious application, DiningByFriends, that uses friends and ratings to provide personalized restaurant recommendations.

In the following notebook, we demonstrate how to use LlamaIndex and Amazon Neptune to build the queries for the DiningByFriends app using Natural Language questions instead of explicitly writing the queries.  


In [2]:
%seed --model property_graph --language gremlin --dataset dining_by_friends --run

Dropdown(description='Source type:', options=('', 'samples', 'custom'), style=DescriptionStyle(description_wid…

Dropdown(description='Data model:', layout=Layout(display='none', visibility='hidden'), options=('', 'property…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Data set:', layout=Layout(display='none', visibility='hidden'), options=(), style=Descri…

...

Dropdown(description='Location:', layout=Layout(display='none', visibility='hidden'), options=('Local', 'S3'),…

FileChooser(path='/home/ec2-user/SageMaker', filename='', title='', show_hidden=False, select_desc='Select', c…

Button(description='Submit', layout=Layout(visibility='hidden'), style=ButtonStyle())

Output()

Output()

## Setting up our LLM

In [3]:
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding


# Setup LlamaIndex to use Claude V3 Sonnet for the LLM
llm = Bedrock(model="anthropic.claude-3-sonnet-20240229-v1:0")

# Create the embedding model, this is required by the Property Graph
embed_model = BedrockEmbedding(model="amazon.titan-embed-text-v1")

In [4]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

## Setting up our GraphStore

In [5]:
from llama_index.graph_stores.neptune import NeptuneDatabasePropertyGraphStore
import graph_notebook as gn

# Retrieve the configuration of the notebook to get the current host
config = gn.configuration.get_config.get_config()

graph_store = NeptuneDatabasePropertyGraphStore(host=config.host)

## Setting up our Index

In [6]:
from llama_index.core import PropertyGraphIndex
index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store
)

## Setting up our Retriever

In [7]:
from llama_index.core.indices.property_graph import CypherTemplateRetriever
from pydantic.v1 import BaseModel, Field

class TemplateParams(BaseModel):
    """Template params for a cypher query."""

    names: list[str] = Field(
        description="A list of person names to use for lookup in a knowledge graph."
    )

# Friends Query
cypher_query = """
MATCH (p1:person)-[:friends]->(p) 
WHERE p1.first_name in $names
RETURN p.first_name"""

retriever = CypherTemplateRetriever(index.property_graph_store, TemplateParams, cypher_query)

## Querying our graph

In [8]:
nodes = retriever.retrieve("Who are Dave's Friends?")

for node in nodes:
    print(node.text)

[{'p.first_name': 'Kelly'}, {'p.first_name': 'Jim'}, {'p.first_name': 'Josh'}, {'p.first_name': 'Hank'}]


In [9]:
# Connected Friends Query
cypher_query = """
MATCH p=(src:person {first_name:$from_person})-[:friends*1..]-(dst:person {first_name:$to_person})
RETURN p LIMIT 1"""

class TemplateParams(BaseModel):
    """Template params for a cypher query."""

    from_person: str = Field(
        description="A person names to start for lookup in a knowledge graph."
    )
    to_person: str = Field(
        description="A person names to end for lookup in a knowledge graph."
    )

retriever = CypherTemplateRetriever(index.property_graph_store, TemplateParams, cypher_query)

nodes = retriever.retrieve("Are Dave and Denise connected?")

for node in nodes:
    print(node.text)

[{'p': [{'~id': '10', '~entityType': 'node', '~labels': ['person'], '~properties': {'person_id': 1, 'last_name': 'Bech', 'first_name': 'Dave'}}, {'~id': '55', '~entityType': 'relationship', '~start': '10', '~end': '30', '~type': 'friends', '~properties': {}}, {'~id': '30', '~entityType': 'node', '~labels': ['person'], '~properties': {'person_id': 5, 'last_name': 'Gorman', 'first_name': 'Kelly'}}, {'~id': '56', '~entityType': 'relationship', '~start': '30', '~end': '45', '~type': 'friends', '~properties': {}}, {'~id': '45', '~entityType': 'node', '~labels': ['person'], '~properties': {'person_id': 8, 'last_name': 'Mande', 'first_name': 'Denise'}}]}]


In [10]:
cypher_query = """
MATCH (p:person {first_name: $from_person})-[:lives]->(c:city)<-[:within]-(r:restaurant)-[:serves]->(cuisine:cuisine)
WHERE cuisine.name IN $cusines
MATCH (r)<-[a:about]-(rev:review)
WITH r, max(rev.rating) AS max_rating
MATCH (r)<-[a:about]-(rev:review)
WHERE rev.rating = max_rating
RETURN r.name, rev.rating
ORDER BY r.name"""

class TemplateParams(BaseModel):
    """Template params for a cypher query."""

    from_person: str = Field(
        description="A person names to start for lookup in a knowledge graph."
    )
    cusines: list[str] = Field(
        description="Cuisine names for restaurant types to lookup in a knowledge graph."
    )

retriever = CypherTemplateRetriever(index.property_graph_store, TemplateParams, cypher_query)

nodes = retriever.retrieve("What restaurants near Dave with a diner or bar cuisine is the highest rated?")

for node in nodes:
    print(node.text)

[{'r.name': 'All Night Long', 'rev.rating': 4}, {'r.name': 'Northern Quench', 'rev.rating': 4}, {'r.name': 'Northern Quench', 'rev.rating': 4}, {'r.name': 'Satiated', 'rev.rating': 4}, {'r.name': 'Satiated', 'rev.rating': 4}, {'r.name': 'Without Chaser', 'rev.rating': 4}]
